In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
print("Training data shape:", x_train.shape)
print("Training labels shape:", y_train.shape)
print("Test data shape:", x_test.shape)
print("Test labels shape:", y_test.shape)

In [ ]:
# Reshaping for CNN (adding channel dimension)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1) / 255.0
x_test  = x_test.reshape(x_test.shape[0], 28, 28, 1) / 255.0
print("Reshaped training data:", x_train.shape)
print("Reshaped test data:", x_test.shape)

In [ ]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [ ]:
model.summary()  # Prints the model architecture, showing the Dropout layers clearly.

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Set up Early Stopping: monitor validation loss, stop if it doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)

history = model.fit(
    x_train,
    y_train,
    epochs=20,  # Increased to 20 to allow early stopping to kick in
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping]
)

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test Accuracy:", test_accuracy)

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Train Accuracy", "Validation Accuracy"])
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Train Loss", "Validation Loss"])
plt.grid(True)
plt.show()

In [ ]:
# Generate predictions for the test dataset
predictions = model.predict(x_test)

# Function to plot images and their predicted/true labels
def plot_predictions(images, labels, preds, num_images=10):
    plt.figure(figsize=(15, 6))
    for i in range(num_images):
        plt.subplot(2, 5, i + 1)
        plt.imshow(images[i].reshape(28, 28), cmap='gray')
        
        predicted_label = np.argmax(preds[i])
        true_label = labels[i]
        
        color = 'green' if predicted_label == true_label else 'red'
        
        plt.title(f"Pred: {predicted_label} | True: {true_label}", color=color)
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Plot first 10 test images
plot_predictions(x_test, y_test, predictions, num_images=10)

### Observation:

After inspecting the dataset shape, the images were reshaped into `(28, 28, 1)` dimensional vectors to be fed into the CNN.

**Architecture:** The `model.summary()` clearly shows the structure, including the **Dropout layers** (`Dropout(0.25)` and `Dropout(0.5)`) added to prevent overfitting.

**Early Stopping:** We set the training to run for up to 20 epochs, but implemented an `EarlyStopping` callback monitoring `val_loss` with a patience of 3. As you can see in the training logs, when the validation loss stops improving for 3 consecutive epochs, the training halts automatically (printing *"Restoring model weights from the end of the best epoch"*), saving time and preventing the model from overfitting on the training data.

**Result:** With Dropout and Early Stopping, the final model yields a highly robust accuracy on testing data, keeping the train and validation losses stable.